In [43]:
import requests

API_KEY = 'VOTRE_CLÉ_API_AMBEE'
latitude = 48.8566   # Exemple : Paris
longitude = 2.3522

url = f'https://api.ambeedata.com/latest/pollen/by-lat-lng'
headers = {
    'x-api-key': '1abda428d8ab2136e2b6e8a85131ff222c3551f66c2e756202c49a27914dec09',
    'Content-type': 'application/json'
}
params = {
    'lat': latitude,
    'lng': longitude
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Erreur: {response.status_code} - {response.text}")

{'message': 'success', 'lat': 48.8566, 'lng': 2.3522, 'data': [{'timezone': 'Europe/Paris', 'Species': {'Grass': {'Grass / Poaceae': 0}, 'Others': 2, 'Tree': {'Alder': 0, 'Birch': 0, 'Cypress': 0, 'Elm': 0, 'Hazel': 0, 'Oak': 0, 'Pine': 0, 'Plane': 0, 'Poplar / Cottonwood': 0}, 'Weed': {'Chenopod': 2, 'Mugwort': 1, 'Nettle': 28, 'Ragweed': 9}}, 'Risk': {'grass_pollen': 'Low', 'tree_pollen': 'Low', 'weed_pollen': 'Moderate'}, 'Count': {'grass_pollen': 0, 'tree_pollen': 0, 'weed_pollen': 42}, 'updatedAt': '2024-09-29T16:00:00.000Z'}]}


In [28]:
print(response)

<Response [200]>


In [8]:
import os
import requests
import json
import logging
from datetime import datetime

# Set up logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Mock the OPENWEATHER_API_KEY for testing purposes
os.environ['OPENWEATHER_API_KEY'] = "1d548abe1933ffd33f6a2d5eed806928"  # Replace with your actual API key if you want to test live

# API request details
url = f"http://api.openweathermap.org/data/2.5/forecast?lat=48.7651&lon=2.2666&appid={os.environ['OPENWEATHER_API_KEY']}&units=metric"

try:
    # Make the request
    response = requests.get(url)
    response.raise_for_status()

    # Parse the response
    data = response.json()

    # Extract forecast data for the next hour
    if "list" in data:
        next_hour_data = data["list"][0]  

        # Formatting the output for easier reading
        next_hour_info = {
            "datetime": next_hour_data.get("dt_txt", "N/A"),
            "temperature": next_hour_data["main"].get("temp", "N/A"),
            "feels_like": next_hour_data["main"].get("feels_like", "N/A"),
            "weather": next_hour_data["weather"][0].get("description", "N/A"),
            "wind_speed": next_hour_data["wind"].get("speed", "N/A"),
            "humidity": next_hour_data["main"].get("humidity", "N/A"),
        }

        # Print the extracted information
        print(json.dumps(next_hour_info, indent=4))

    else:
        logger.error("No forecast data available.")

except requests.exceptions.RequestException as e:
    logger.error(f"An error occurred while making the request: {e}")

{
    "datetime": "2024-10-07 09:00:00",
    "temperature": 15.62,
    "feels_like": 15.25,
    "weather": "overcast clouds",
    "wind_speed": 3.32,
    "humidity": 77
}


In [11]:
import os

import requests
import json
import logging
from typing import Dict, Any


# Set up logging
logger = logging.getLogger(__name__)

class WeatherData:
    def __init__(self, lat: float = 48.7651, lon: float = 2.2666):
        self.lat = lat
        self.lon = lon
        self.api_key = os.getenv('OPENWEATHER_API_KEY')

    def set_location(self, lat: float, lon: float):
        self.lat = lat
        self.lon = lon

    def get_weather_forecast(self) -> Dict[str, Any]:
        if not self.api_key:
            logger.error("OpenWeather API key is not set.")
            return self._default_weather_response()

        url = f"http://api.openweathermap.org/data/2.5/forecast?lat={self.lat}&lon={self.lon}&appid={self.api_key}&units=metric"

        try:
            # Send the request
            response = requests.get(url)
            response.raise_for_status()
            weather_data = response.json()

            # Extract +1 hour and +24 hours forecast data
            list_data = weather_data.get('list', [])
            if len(list_data) < 24:
                logger.error("Not enough forecast data available to extract +1H and +24H data.")
                return self._default_weather_response()

            next_hour_data = list_data[0]
            next_24_hour_data = list_data[24]  # Assuming each forecast is at 3-hour intervals, index 7 represents approximately +24 hours

            next_hour_info = {
                "datetime": next_hour_data.get("dt_txt", "N/A"),
                "temperature": next_hour_data["main"].get("temp", "N/A"),
                "feels_like": next_hour_data["main"].get("feels_like", "N/A"),
                "weather": next_hour_data["weather"][0].get("description", "N/A"),
                "wind_speed": next_hour_data["wind"].get("speed", "N/A"),
                "humidity": next_hour_data["main"].get("humidity", "N/A"),
            }

            next_24_hour_info = {
                "datetime": next_24_hour_data.get("dt_txt", "N/A"),
                "temperature": next_24_hour_data["main"].get("temp", "N/A"),
                "feels_like": next_24_hour_data["main"].get("feels_like", "N/A"),
                "weather": next_24_hour_data["weather"][0].get("description", "N/A"),
                "wind_speed": next_24_hour_data["wind"].get("speed", "N/A"),
                "humidity": next_24_hour_data["main"].get("humidity", "N/A"),
            }

            # Return formatted weather data for +1 hour and +24 hours
            return {
                'plus1H': next_hour_info,
                '+24H': next_24_hour_info
            }

        except requests.RequestException as e:
            logger.error(f"HTTP Request failed: {e}")
            return self._default_weather_response()

# Test 
weather_data = WeatherData()
weather_data.get_weather_forecast()
weather_data.set_location(48.8566, 2.3522)
weather_data.get_weather_forecast()
print(weather_data.get_weather_forecast())


{'plus1H': {'datetime': '2024-10-07 09:00:00', 'temperature': 16.37, 'feels_like': 16.05, 'weather': 'overcast clouds', 'wind_speed': 2.8, 'humidity': 76}, '+24H': {'datetime': '2024-10-10 09:00:00', 'temperature': 12.75, 'feels_like': 12.27, 'weather': 'light rain', 'wind_speed': 6.57, 'humidity': 84}}


In [ ]:
import requests
# import keras

